In [3]:
# Francisco Dominguez Mateos
# 25/08/2021
# Voice Stress detection PC

In [4]:
import glob
import time
import numpy as np
#np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [5]:
path_base="/home/francisco/datasets/datasets/sound/voice/stress"

In [6]:
def getLabel(s):
    if s=="TRUE":
        return 1.0
    if s=="FALSE":
        return 0.0
    if s=="PC":
        return 2.0
def isRightLabel(s):
    if s=="TRUE":
        return True
    if s=="FALSE":
        return True
    if s=="PC":
        return True
    return False

In [7]:
data=[]
labels=[]
file_txt=path_base+"/*.csv"
for filepath in glob.glob(file_txt):
    print("Processing file: {}".format(filepath)) 
    with open(filepath) as fp:  
        line = fp.readline()
        head_list=line.split(";")
        #for i,head in enumerate(head_list):
        #    print(i,head)
        cnt = 1
        line = fp.readline()
        while line:
            data_row=[]
            line_list=line.split(";")
            #print("Line {}: {} {} {}".format(cnt, line_list[0], line_list[1], line_list[2]))
            line = fp.readline()
            cnt += 1
            if not isRightLabel(line_list[0]):
                continue
            for i,datum in enumerate(line_list):
                #print(i,head_list[i],"=",datum)
                if i>1:
                    data_row.append(float(datum))
                if i==0:
                    labels.append(getLabel(datum))
            data.append(data_row)
data_np=np.array(data)
labels_np=np.array(labels)
print(data_np.shape)
print(data_np)
print(labels_np.shape)
print(labels_np)

Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Males.csv
Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Females.csv
(76, 68)
[[126.278 0.014 0.029 ... 15.681 0.870 0.012]
 [142.901 0.005 0.019 ... 10.834 0.936 0.008]
 [117.189 0.005 0.014 ... 11.765 0.967 0.006]
 ...
 [231.855 0.016 0.014 ... 10.118 0.933 0.018]
 [202.053 0.020 0.024 ... 15.960 0.903 0.013]
 [235.090 0.014 0.023 ... 8.176 0.936 0.291]]
(76,)
[1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.000 2.000 2.000 2.000
 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 2.000 2.000 2.000 2.000 2.000 2.000
 2.000 2.000 2.000 2.000]


In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
ss=StandardScaler()
data_ss=ss.fit_transform(data_np)

print(data_ss.shape)
print(data_ss)

(76, 68)
[[-0.429 -0.118 -0.110 ... 0.630 -1.390 -0.450]
 [-0.210 -0.254 -0.165 ... -0.263 -0.010 -0.545]
 [-0.548 -0.254 -0.193 ... -0.092 0.638 -0.593]
 ...
 [0.959 -0.087 -0.193 ... -0.395 -0.073 -0.308]
 [0.568 -0.027 -0.138 ... 0.682 -0.700 -0.426]
 [1.002 -0.118 -0.143 ... -0.754 -0.010 6.185]]


In [10]:
def one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k """
    return np.array(x[:, None] == np.arange(k), dtype)

In [11]:
num_classes=int(np.max(labels_np)+1)
print(num_classes)

3


In [12]:
labels_onehot=one_hot(labels_np,num_classes)
print(labels_onehot.shape)

(76, 3)


In [13]:
import numpy as onp
import jax.numpy as np
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.experimental import stax
from jax.experimental.stax import (BatchNorm, Conv, Dense, Flatten, Dropout,
                                   Relu, LogSoftmax)

In [14]:
# Import some additional JAX and dataloader helpers
from jax.scipy.special import logsumexp
from jax.experimental import optimizers

In [15]:
#Test if JAX is using CPU or GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [16]:
# Generate key which is used to generate random numbers
key = random.PRNGKey(1)

In [29]:
hidden=25
dropout_rate=0.50
def makeNet(num_classes,hidden,dropout_rate,mode="train"):
    init_fun, net = stax.serial(
        Dense(hidden),
        #BatchNorm(axis=0),
        Relu,
        #Dropout(dropout_rate,mode=mode),
        #Dense(hidden),
        #BatchNorm(axis=0),
        #Relu,
        Dropout(dropout_rate,mode=mode),
        Dense(num_classes),
        Dense(num_classes),
        LogSoftmax)
    return init_fun,net

In [30]:
init_fun, net=makeNet(num_classes,hidden,dropout_rate)
input_shape=(-1,)+ data_ss.shape[1:]
print("input_shape=",input_shape)
output_shape, params = init_fun(key, input_shape)
print("ouput_shape=",output_shape)
_,netTest=makeNet(num_classes,hidden,dropout_rate,mode='test')

input_shape= (-1, 68)
ouput_shape= (-1, 3)


In [31]:
def loss(params, data, targets,key):
    preds = net(params, data,rng=key)
    return -np.sum(preds * targets)

def predict(params,data,key):
    preds=netTest(params,data,rng=key)
    return np.exp(preds)

def accuracy(p,targets):
    target_class    = np.argmax(targets, axis=1)
    predicted_class = np.argmax(p      , axis=1)
    acc_total       = np.sum(predicted_class == target_class)
    return acc_total/p.shape[0]


In [32]:
step_size = 1e-3
opt_init, opt_update, get_params = optimizers.adam(step_size)
opt_state = opt_init(params)
num_epochs = 2

In [33]:
@jit
def update(params, x, y, opt_state,key):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(loss)(params, x, y,key)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [34]:
def run_training_loop(data_ss,labels_onehot,num_epochs, opt_state,verbose=False):
    """ Implements a learning loop over epochs. """
    # Get the initial set of parameters
    params = get_params(opt_state)

    # Loop over the training epochs
    for epoch in range(num_epochs):
        start_time = time.time()
        params, opt_state, loss = update(params, data_ss, labels_onehot, opt_state,key)
        epoch_time = time.time() - start_time
        if verbose: print("Epoch {} | T: {:0.2f} | loss: {:0.3f} ".format(epoch+1, epoch_time,
                                                                    loss))
    return loss,params

In [35]:
l,params=run_training_loop(data_ss,labels_onehot,2000,opt_state,False)
print(l)

0.4337315


In [36]:
def _numpy_delete(x, idx):
    """
    Gets the subarray from `x` where data from index `idx` on the first axis is removed.
    """
    # NB: numpy.delete is not yet available in JAX
    mask = np.arange(x.shape[0] - 1) < idx
    return np.where(mask.reshape((-1,) + (1,) * (x.ndim - 1)), x[:-1], x[1:])
def allButOne(x,y,i):
    global key
    oneX=x[i:i+1] #this return a (1,N) shape vs x[i] gives (N,) shape
    oneY=y[i:i+1]
    allXBut=_numpy_delete(x,i)
    allYBut=_numpy_delete(y,i)
    rng,key=random.split(key)
    idxs=allXBut.shape[0]
    suffleIdx=random.permutation(rng,idxs)
    allXBut=allXBut[suffleIdx]
    allYBut=allYBut[suffleIdx]
    return allXBut,allYBut,oneX,oneY

In [37]:
allXBut,allYBut,oneX,oneY=allButOne(data_ss,labels_onehot,1)
print(allXBut.shape)
print(allYBut.shape)
print(oneX.shape)
print(oneY.shape)
oneY

(75, 68)
(75, 3)
(1, 68)
(1, 3)


array([[0.000, 1.000, 0.000]], dtype=float32)

In [38]:
accT=0
for i in range(data_ss.shape[0]):
    allXBut,allYBut,oneX,oneY=allButOne(data_ss,labels_onehot,i)
    loss,params=run_training_loop(allXBut,allYBut,20000,opt_state)
    p=predict(params,oneX,key)
    acc=accuracy(p,oneY)
    accT+=acc
    print("i=",i,p,oneY,"acc={0:0.3f} accMean={1:0.3f} loss={2:0.3f}".format(acc,accT/(i+1),loss))
print("accMean={0:0.3f}".format(accT/(data_ss.shape[0])))

i= 0 [[1.000 0.000 0.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.000 loss=0.000
i= 1 [[1.000 0.000 0.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.000 loss=0.000
i= 2 [[0.994 0.006 0.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.000 loss=0.000
i= 3 [[0.034 0.966 0.000]] [[0.000 1.000 0.000]] acc=1.000 accMean=0.250 loss=0.000
i= 4 [[0.997 0.003 0.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.200 loss=0.000
i= 5 [[0.000 0.000 1.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.167 loss=0.000
i= 6 [[0.995 0.005 0.001]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.143 loss=0.000
i= 7 [[0.000 0.000 1.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.125 loss=0.000
i= 8 [[0.990 0.010 0.000]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.111 loss=0.000
i= 9 [[0.000 0.001 0.998]] [[0.000 1.000 0.000]] acc=0.000 accMean=0.100 loss=-0.000
i= 10 [[0.410 0.589 0.001]] [[1.000 0.000 0.000]] acc=0.000 accMean=0.091 loss=0.000
i= 11 [[0.002 0.998 0.000]] [[1.000 0.000 0.000]] acc=0.000 accMean=0.083 